# Part 1: Data Preprocessing

Dataset link: [Predicting Churn for Bank Customers](https://www.kaggle.com/datasets/adammaus/predicting-churn-for-bank-customers?select=Churn_Modelling.csv)

## Importing the libraries

In [194]:
import sklearn
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import ceil
import random
from xgboost import XGBClassifier
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, MaxPooling2D, Flatten
from keras.losses import BinaryCrossentropy, SparseCategoricalCrossentropy
from keras.optimizers import Adam

## Dealing dataset

In [195]:
dataset = pd.read_csv('train.csv')

In [196]:
dataset

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15633608,Black,641,France,Male,33,2,146193.60,2,1,1,55796.83,1
1,1,15742323,Barese,541,France,Male,39,7,0.00,2,1,0,19823.02,0
2,2,15760244,Ives,590,France,Female,76,5,160979.68,1,0,1,13848.58,0
3,3,15600974,He,516,Spain,Male,50,5,0.00,1,0,1,146145.93,1
4,4,15663234,Bishop,508,France,Female,60,7,143262.04,1,1,1,129562.74,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,6995,15584928,Ugochukwutubelum,594,Germany,Female,32,4,120074.97,2,1,1,162961.79,0
6996,6996,15647111,White,794,Spain,Female,22,4,114440.24,1,1,1,107753.07,0
6997,6997,15574372,Hoolan,738,France,Male,35,5,161274.05,2,1,0,181429.87,0
6998,6998,15664035,Parsons,590,Spain,Female,38,9,0.00,2,1,1,148750.16,0


In [197]:
dataset = dataset.drop(columns=['id', 'CustomerId', 'Surname'])

In [198]:
dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,641,France,Male,33,2,146193.60,2,1,1,55796.83,1
1,541,France,Male,39,7,0.00,2,1,0,19823.02,0
2,590,France,Female,76,5,160979.68,1,0,1,13848.58,0
3,516,Spain,Male,50,5,0.00,1,0,1,146145.93,1
4,508,France,Female,60,7,143262.04,1,1,1,129562.74,0


In [199]:
dataset.shape

(7000, 11)

In [200]:
# detect missing data
dataset.isnull().values.any()

False

In [201]:
dataset.select_dtypes(include='object').columns

Index(['Geography', 'Gender'], dtype='object')

In [202]:
dataset['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [203]:
dataset['Gender'].unique()

array(['Male', 'Female'], dtype=object)

In [204]:
dataset = pd.get_dummies(data=dataset, drop_first=True)

In [205]:
dataset

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,641,33,2,146193.60,2,1,1,55796.83,1,0,0,1
1,541,39,7,0.00,2,1,0,19823.02,0,0,0,1
2,590,76,5,160979.68,1,0,1,13848.58,0,0,0,0
3,516,50,5,0.00,1,0,1,146145.93,1,0,1,1
4,508,60,7,143262.04,1,1,1,129562.74,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6995,594,32,4,120074.97,2,1,1,162961.79,0,1,0,0
6996,794,22,4,114440.24,1,1,1,107753.07,0,0,1,0
6997,738,35,5,161274.05,2,1,0,181429.87,0,0,0,1
6998,590,38,9,0.00,2,1,1,148750.16,0,0,1,0


In [206]:
x = dataset.drop(columns='Exited').values

In [207]:
y = dataset['Exited'].values

In [208]:
x.shape, y.shape

((7000, 11), (7000,))

## Splitting the training data and testing data

In [209]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1)

In [210]:
# x_train = x
# y_train = y

In [211]:
x_train.shape, x_test.shape

((6300, 11), (700, 11))

## Feature Scaling

In [212]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

# Part 2: Building the model

In [213]:
models_result = []

In [214]:
def accuracy(y_hat, y_test, threshold=0.5):
    y_hat = ((y_hat >= threshold)*1).astype('int64')
    return (y_test == y_hat).sum() / len(y_hat)

## Logistic regression

In [215]:
accuracy_lr = []
threshold_lr = []

In [216]:
## config
def get_model(config):
    layers = [
        Dense(units=u, activation='relu') for u in config
    ]
    layers.append(Dense(units=1, activation='sigmoid'))
    return Sequential(layers)

In [217]:
def train_eval(model, x_train, x_test, y_train, y_test):
    y_hat = model.predict(x_test).flatten()
    
    acc_max = 0
    threshold_optim = 0.5

    thresholds = np.arange(0.1,1, step = 0.01)
    for i, threshold in enumerate(thresholds):
        acc = accuracy(y_hat, y_test, threshold)
        if acc_max < acc:
            acc_max = acc
            threshold_optim = thresholds[i]

    accuracy_lr.append(acc_max)
    threshold_lr.append(threshold_optim)
    return acc_max

In [218]:
models_classifier_lr = [
    get_model(config) for config in 
    [
        [25, 15, 10],
        [87, 71, 42, 14],
        [55, 51, 45, 33, 20, 2],
        [53, 47, 14, 13, 9, 8, 5],
        [39, 34, 29, 28, 17, 6, 5]
    ]
]

# [model.compile(optimizer=Adam(learning_rate=1e-3), loss=SparseCategoricalCrossentropy(from_logits=True)) for model in models]
[model.compile(loss=BinaryCrossentropy()) for model in models_classifier_lr]

[model.fit(x_train, y_train, epochs=100) for model in models_classifier_lr]

[train_eval(model, x_train, x_test, y_train, y_test) for model in models_classifier_lr]

Epoch 1/100
197/197 [==============================] - 1s 2ms/step - loss: 0.6166
Epoch 2/100
197/197 [==============================] - 0s 2ms/step - loss: 0.4628
Epoch 3/100
197/197 [==============================] - 0s 3ms/step - loss: 0.4300
Epoch 4/100
197/197 [==============================] - 0s 2ms/step - loss: 0.4156
Epoch 5/100
197/197 [==============================] - 0s 1ms/step - loss: 0.4064
Epoch 6/100
197/197 [==============================] - 0s 2ms/step - loss: 0.3981
Epoch 7/100
197/197 [==============================] - 0s 2ms/step - loss: 0.3892
Epoch 8/100
197/197 [==============================] - 0s 2ms/step - loss: 0.3799
Epoch 9/100
197/197 [==============================] - 0s 2ms/step - loss: 0.3689
Epoch 10/100
197/197 [==============================] - 0s 2ms/step - loss: 0.3570
Epoch 11/100
197/197 [==============================] - 0s 2ms/step - loss: 0.3465
Epoch 12/100
197/197 [==============================] - 0s 2ms/step - loss: 0.3405
Epoch 13/100


[0.8442857142857143,
 0.8328571428571429,
 0.8171428571428572,
 0.8128571428571428,
 0.8071428571428572]

In [219]:
accuracy_lr

[0.8442857142857143,
 0.8328571428571429,
 0.8171428571428572,
 0.8128571428571428,
 0.8071428571428572]

In [220]:
acc_max = 0
i = 0
for acc in accuracy_lr:
    if acc_max < acc:
        acc_max = acc
        id_optim = i
    i = i + 1

In [221]:
results = pd.DataFrame([['Logistic regression', accuracy_lr[id_optim]]], columns=['Model', 'Accuracy'])
results

,Model,Accuracy
0,Logistic regression,0.844286


In [222]:
models_result.append([models_classifier_lr[id_optim], accuracy_lr[id_optim], threshold_lr[id_optim]])

## XGBoost

In [223]:
classifier_xgb = XGBClassifier()
classifier_xgb.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [224]:
y_hat = classifier_xgb.predict(x_test).flatten()

In [225]:
accuracy_xgb = accuracy(y_hat, y_test)

In [226]:
result = pd.DataFrame([['XGBoost', accuracy_xgb]], columns=['Model', 'Accuracy'])
results = pd.concat([results, result], ignore_index=True)
results

,Model,Accuracy
0,Logistic regression,0.844286
1,XGBoost,0.844286


In [227]:
models_result.append([classifier_xgb, accuracy_xgb, 0.5])

# Part 3: Answer

## Dealing dataset

In [228]:
dataset_ans = pd.read_csv('test.csv')

In [229]:
dataset_ans = dataset_ans.drop(columns=['id', 'CustomerId', 'Surname'])

In [230]:
dataset_ans = pd.get_dummies(data=dataset_ans, drop_first=True)

In [231]:
dataset_ans

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
0,597,35,8,131101.04,1,1,1,192852.67,1,0,0
1,523,40,2,102967.41,1,1,0,128702.10,0,0,0
2,706,42,8,95386.82,1,1,1,75732.25,0,1,0
3,788,32,4,112079.58,1,0,0,89368.59,0,0,1
4,706,38,5,163034.82,2,1,1,135662.17,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
2995,530,41,4,0.00,2,0,1,147606.71,0,1,0
2996,639,77,6,80926.02,2,1,1,55829.25,0,0,1
2997,685,40,2,168001.34,2,1,1,167400.29,0,0,1
2998,692,69,10,154953.94,1,1,1,70849.47,0,0,1


In [232]:
x_ans = dataset_ans.values

In [233]:
x_ans.shape

(3000, 11)

## Feature Scaling

In [234]:
x_ans = sc.transform(x_ans)

## Answer

In [235]:
def standardized_y(y_hat, threshold = 0.9):
    y_hat = ((y_hat >= threshold)*1).astype('int64')
    return y_hat

In [236]:
results

,Model,Accuracy
0,Logistic regression,0.844286
1,XGBoost,0.844286


In [237]:
acc_max = 0
id_optim = 0
for i in range(results.shape[0]):
    if acc_max < results['Accuracy'][i]:
        acc_max = results['Accuracy'][i]
        id_optim = i

In [238]:
models_result[id_optim][0]

In [239]:
y_ans = models_result[id_optim][0].predict(x_ans).flatten()
y_ans = standardized_y(y_ans, models_result[id_optim][2])

94/94 [==============================] - 0s 1ms/step


In [240]:
[print(f"{y_ans[i]:.0f}") for i in range(len(y_ans))]

0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
1
1
0
0
1
1
0
1
0
0
0
1
0
0
0
1
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
1
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
1
1
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
1
0
0
0
0
0
0
0
0
0
1
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
1
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
1
0
0
0
0
0
1
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
1
1
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,